#### Importieren der notwendigen Bibliotheken

Wir wollen die Tensorflow Version 2.0 verwenden und geben diese daher spezifisch an. 



In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

#### Laden des MNIST Datensatzes

Als Erstes wollen wir den Datensatz wie im Video "Laden und Bearbeiten des MNIST Datensatz" laden.

In [3]:
(train_images, train_labels), (test_images, test_labels) = keras. \
  datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


Wir errinnern uns, dass die Pixelwerte noch nicht in normaliserter Form vorliegen. Wir normalisieren diese also zunächst indem wir durch den maximalen Pixelwert 255 teilen:

In [0]:
train_images = train_images / 255.0
test_images = test_images / 255.0

Zuletzt hatten wir noch das Problem, dass die Labels des Datensatzes einfach nur Zahlen waren.
- Das Bild der handgeschriebenen 5 hat das Label `5`.
- Wir hätten in diesem Fall jedoch gerne den Vektor `[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]` als Label, welcher der von unserem Netz gewünschten Ausgabe entspricht. Dieser Vektor hat nur an Stelle 5 (beginnend bei 0) eine 1.

Für detailliertere Erklärungen sei an dieser Stelle wieder auf das Video "Laden und Bearbeiten des MNIST Datensatzes" verwiesen.

In [0]:
total_classes = 10
train_vec_labels = keras.utils.to_categorical(train_labels, total_classes)
test_vec_labels = keras.utils.to_categorical(test_labels, total_classes)

#### Entwurf der Netze

Nun haben wir die Eingabedaten normalisiert und die Labels als Vektoren vorliegen. Wir können also endlich anfangen die Netze für die Erkennung der handgeschriebenen Zahlen zu bauen! :) 

Wir wollen dafür verschiedene, sehr einfache Netze mit 3 Schichten definieren (Eingabelayer, Hidden Layer und Ausgabelayer):
- Als **Input-Layer** verwenden wir einen `keras.layers.Flatten` Layer, der die 28x28 Matrizen, die wir als Eingaben erhalten auf $28\cdot 28 = 784$ Neuronen verteilt
- Als nächstes verwenden wir für den **Hidden-Layer** einen `keras.layers.Dense` Layer mit 128 Neuronen, wobei wir 128 als eine gute Anzahl empfinden
- Als **Output-Layer** verwenden wir einen `keras.layers.Dense` Layer mit 10 Neuronen, da wir 10 Klassen (Ziffern von 0-9) erkennen wollen

Wir definieren die einzelnen Netze mit den unterschiedlichen Aktivierungsfunktionen, um diese anschließend miteinander vergleichen zu können.


In [0]:
# model_no_activation = keras.Sequential([
#     keras.layers.Flatten(input_shape=(28, 28)),
#     keras.layers.Dense(128), # , activation='sigmoid'),
#     keras.layers.Dense(10), #, activation='sigmoid')
# ])

model_relu = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='sigmoid')
])

model_linear = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='linear'),
    keras.layers.Dense(10, activation='linear')
])

model_sigmoid = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='sigmoid'),
    keras.layers.Dense(10, activation='sigmoid')
])

model_tanh = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='tanh'),
    keras.layers.Dense(10, activation='tanh')
])

models = [model_relu, model_linear,
          model_sigmoid,model_tanh]

#### Kompilieren der Netze

Nach dem wir unsere Netze definiert haben, müssen wir sie *kompilieren*, bevor wir mit dem Training beginnen können.

In diesem Schritt legen wir wichtige Parameter für die Trainingsphase fest:
- Der **Optimizer** ist der im Training verwendete Lernalgorithmus zur Verbesserung des Netzes. In der letzen Woche haben wir ja bereits *Gradient Descent* und dessen Optimierung *Stochastic Gradient Descent* (SGD, siehe *Deep Dive: Backpropagation*) kennengelernt.
- Der **Loss** ist die verwendete Kostenfunktion. Ziel während des Trainings ist es, diese zu minimieren. Wir haben in Woche 1 bereits die Quadratische Fehlerfunktion (*Squared Error*) kennengelernt.
- Die **Metrics** sind die während des Trainings ausgewerteten Metrics. Bei allen Klassifikationsproblemen interessiert uns hier die `"accuracy"`.

In diesem Beispiel verwenden wir 
- Den *Stochastic Gradient Descent* (`"sgd"`) Lernalgorithmus als unseren Optimizer.
- Die `"mean_squared_error"` Kostenfunktion, welche im Vergleich zur normalen *Squared Error* Kostenfunktion nicht die Summe, sondern den Mittelwert der Fehler der Ausgabeneuronen berechnet.

In [7]:
[
  model.compile(
      optimizer='sgd',
      loss='mean_squared_error',
      metrics=['accuracy']
  ) for model in models
]

[None, None, None, None]

#### Trainieren der Netze

Jetzt können wir endlich unser Netz tranieren. Dazu verwenden wir die `fit` Methode und übergeben unsere Trainingsbilder als Eingaben mit den dazugehörigen Labels als gewünschte Ausgaben. Die Anzahl der `epochs` gibt an, wie oft das Netz das gesamte Trainingsset zu sehen bekommt. Erhöhen wir die Anzahl der Epochen lassen wir unser Netz länger lernen.  

In [8]:
epochs=15
[
 model.fit(
    train_images,
    train_vec_labels,
    epochs=epochs,
    verbose=True
  ) for model in models
]

Train on 60000 samples
Epoch 1/15
60000/60000 [==============================] - 8s 128us/sample - loss: 0.1043 - accuracy: 0.2860
Epoch 2/15
60000/60000 [==============================] - 5s 87us/sample - loss: 0.0783 - accuracy: 0.5271
Epoch 3/15
60000/60000 [==============================] - 5s 86us/sample - loss: 0.0679 - accuracy: 0.6165
Epoch 4/15
60000/60000 [==============================] - 5s 86us/sample - loss: 0.0587 - accuracy: 0.6838
Epoch 5/15
60000/60000 [==============================] - 5s 86us/sample - loss: 0.0516 - accuracy: 0.7475
Epoch 6/15
60000/60000 [==============================] - 5s 86us/sample - loss: 0.0463 - accuracy: 0.7878
Epoch 7/15
60000/60000 [==============================] - 5s 85us/sample - loss: 0.0423 - accuracy: 0.8102
Epoch 8/15
60000/60000 [==============================] - 5s 86us/sample - loss: 0.0391 - accuracy: 0.8244
Epoch 9/15
60000/60000 [==============================] - 5s 85us/sample - loss: 0.0366 - accuracy: 0.8353
Epoch 10/15
6

#### Evaluieren der Netze

Bisher hat die Netze nur Trainingsbilder gesehen und damit gelernt. Ziel ist es, mit unseren Netzen, neue Bilder von handgeschriebenen Ziffern zu erkennen. Dafür gibt es die Testdaten, mit denen wir unsere Netze nun auf die Genauigkeit bei ungesehenen Daten überprüfen wollen.

In [9]:
_, result_relu = model_relu.evaluate(test_images, test_vec_labels)
_, result_linear = model_linear.evaluate(test_images, test_vec_labels)
_, result_sigmoid = model_sigmoid.evaluate(test_images, test_vec_labels)
_, result_tanh = model_tanh.evaluate(test_images, test_vec_labels)

10000/10000 [==============================] - 1s 89us/sample - loss: 0.0307 - accuracy: 0.9108


#### Übersichliche Darstellung der Ergebnisse

Die Darstellung der Ergebnisse mit PrettyTable dient lediglich der Übersichtlichkeit und ist für den Kurs an dieser Stelle nicht weiter wichtig.

In [10]:
from prettytable import PrettyTable
tbl = PrettyTable()
tbl.field_names = ["Activation function", f"Accurracy (after {epochs} epochs)"]
tbl.add_row(["Tanh", result_tanh])
tbl.add_row(["model_relu", result_relu])
tbl.add_row(["Linear", result_linear])
tbl.add_row(["Sigmoid", result_sigmoid])
print(tbl)

+---------------------+-----------------------------+
| Activation function | Accurracy (after 15 epochs) |
+---------------------+-----------------------------+
|         Tanh        |            0.9108           |
|      model_relu     |            0.8791           |
|        Linear       |            0.8599           |
|       Sigmoid       |            0.6645           |
+---------------------+-----------------------------+
